# Recorte 1985 - 2002 -> Estratégia

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./Video_Games_Sales_as_at_22_Dec_2016.csv')
df.dropna(subset=['Publisher'], inplace=True)
df = df.loc[~df['Critic_Score'].isna() & ~df['User_Score'].isna() & ~df['Rating'].isna()]

In [3]:
df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E


In [4]:
df_recortado = df.loc[(df['Year_of_Release'] >= 1985) & (df['Year_of_Release'] <= 2002) & (df['Genre'] == 'Strategy')] 

In [5]:
df_recortado.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
1128,Pikmin,GC,2001.0,Strategy,Nintendo,0.78,0.25,0.56,0.04,1.63,89.0,39.0,8.7,121.0,Nintendo,E
1597,RollerCoaster Tycoon 2,PC,2002.0,Strategy,Atari,1.19,0.05,0.00,0.00,1.25,74.0,22.0,8.5,135.0,Chris Sawyer,E
2172,Kessen,PS2,2000.0,Strategy,Electronic Arts,0.27,0.21,0.41,0.07,0.95,75.0,20.0,8.4,12.0,Koei,T
2919,Advance Wars,GBA,2001.0,Strategy,Nintendo,0.50,0.19,0.00,0.01,0.70,92.0,28.0,9.0,95.0,Intelligent Systems,E
4111,Theme Park Roller Coaster,PS2,2000.0,Strategy,Electronic Arts,0.23,0.18,0.00,0.06,0.48,82.0,11.0,7.9,17.0,Bullfrog Productions,E


In [6]:
df_recortado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 1128 to 16210
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             31 non-null     object 
 1   Platform         31 non-null     object 
 2   Year_of_Release  31 non-null     float64
 3   Genre            31 non-null     object 
 4   Publisher        31 non-null     object 
 5   NA_Sales         31 non-null     float64
 6   EU_Sales         31 non-null     float64
 7   JP_Sales         31 non-null     float64
 8   Other_Sales      31 non-null     float64
 9   Global_Sales     31 non-null     float64
 10  Critic_Score     31 non-null     float64
 11  Critic_Count     31 non-null     float64
 12  User_Score       31 non-null     float64
 13  User_Count       31 non-null     float64
 14  Developer        31 non-null     object 
 15  Rating           31 non-null     object 
dtypes: float64(10), object(6)
memory usage: 4.1+ KB


In [25]:
df_recortado.Year_of_Release.value_counts()

2002.0    455
2001.0    256
2000.0    102
1999.0     30
1998.0     25
1997.0     13
1996.0      7
1994.0      1
1985.0      1
1992.0      1
1988.0      1
Name: Year_of_Release, dtype: int64